In [1]:
#import libraries
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
from create_tables import *
import json

In [2]:
#creating database
create_database()

(<cursor object at 0x7fda8823f618; closed: 0>,
 <connection object at 0x7fda87b9f3d8; dsn: 'user=student password=xxx dbname=sparkifydb host=127.0.0.1', closed: 0>)

In [3]:
#autocommit = true
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
conn.set_session(autocommit=True)

In [4]:
#create get_file function
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [5]:
#setting variables for song_path and song_file 
song_path = 'data/song_data'
song_files = get_files(song_path)


In [6]:
#grabbing the first record
filepath = song_files[0]
filepath

'/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json'

In [7]:
#readding the first record
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


In [8]:
#grabbing columns for 'songs' table
song_data = list(df[['song_id','title','artist_id','year','duration']].values[0])
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

In [9]:
#created 'songs' table and inserted 'song_data' values in the songs table
cur.execute(song_table_create)
cur.execute(song_table_insert, song_data)
conn.commit()

In [10]:
#grabbing columns for 'artist' table
artist_data = list(df[['artist_id','artist_name','artist_location','artist_longitude','artist_latitude']].values[0])
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

In [11]:
#create artists table and insert the first record from the list above
cur.execute(artist_table_create)
cur.execute(artist_table_insert, artist_data)
conn.commit()

In [12]:
log_path = 'data/log_data'
log_files = get_files(log_path)
log_filepath = log_files[0]
df_log = pd.read_json(log_filepath, lines=True)
df_log.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [13]:
#filtering only page = 'NextSong'
df_log_filter = df_log.loc[df_log['page'] == 'NextSong']
df_log_filter.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Shell Shock,200,1543540121796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
2,Morcheeba,Logged In,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Women Lose Weight (Feat: Slick Rick),200,1543540368796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
3,Maroon 5,Logged In,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Won't Go Home Without You,200,1543540625796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Train,Logged In,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,1049,Hey_ Soul Sister,200,1543540856796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73


In [14]:
#extract field 'ts' and convert from int to datetime
t = pd.to_datetime(df_log_filter['ts'], unit = 'ms')
t.head()

0   2018-11-30 00:22:07.796
1   2018-11-30 01:08:41.796
2   2018-11-30 01:12:48.796
3   2018-11-30 01:17:05.796
4   2018-11-30 01:20:56.796
Name: ts, dtype: datetime64[ns]

In [15]:
#extract detail information from t and store in time_data, also created labels for the upcoming dataframe
time_data = (pd.to_datetime(df_log_filter['ts'], unit = 'ms'),t.dt.hour, t.dt.day,t.dt.weekofyear,t.dt.month,t.dt.year,t.dt.weekday)
column_labels = ('datetime','hour','day','weekofyear','month','year','weekday')

In [16]:
#zip the 2 fields as dictionary and create dataframe
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))
time_df.head()

,datetime,hour,day,weekofyear,month,year,weekday
0,2018-11-30 00:22:07.796,0,30,48,11,2018,4
1,2018-11-30 01:08:41.796,1,30,48,11,2018,4
2,2018-11-30 01:12:48.796,1,30,48,11,2018,4
3,2018-11-30 01:17:05.796,1,30,48,11,2018,4
4,2018-11-30 01:20:56.796,1,30,48,11,2018,4


In [17]:
#create time table and run all fields into the time table
cur.execute(time_table_create)
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

In [18]:
#grabbed the columns needed for the users table
user_df = df_log_filter[['userId', 'firstName','lastName','gender','level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,91,Jayden,Bell,M,free
1,73,Jacob,Klein,M,paid
2,73,Jacob,Klein,M,paid
3,73,Jacob,Klein,M,paid
4,73,Jacob,Klein,M,paid


In [19]:
#created users table and inserted the records
cur.execute(user_table_create)
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

In [20]:
#created songplay table
cur.execute(songplay_table_create)

In [21]:
#ran sql query to join songs and artist table to get the songid and artist id, then inserted all the data required into the songplay table
for index, row in df_log.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (pd.to_datetime(row.ts, unit = 'ms'), row.userId, row.level, songid, artistid,row.sessionId, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

In [22]:
conn.close()